# Assignment 2 - Randomized Optimization - Part I
## student: hyao66

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mlrose

In [ ]:
# import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import time
import random
import itertools
import math

## part 1: Apply four search techniques (RHC, SA, GA, MIMIC) to three optimization problems

In [ ]:
def fill_array(max_iters, curve):
    max_value = max(curve)
    
    empty_array = np.empty(max_iters)
    empty_array[:] = max_value
    
    len_curve = len(curve)
    
    if len_curve >= max_iters:
        empty_array[0:max_iters] = curve[0:max_iters]
    else:
        empty_array[0:len_curve] = curve[0:len_curve]
    
    return empty_array, len_curve

In [ ]:
##############################################
# define a Four Peaks fitness function object
##############################################

fitness = mlrose.FourPeaks() #with default t_pct = 0.1
# define a optimization problem object
problem = mlrose.DiscreteOpt(length = 100, fitness_fn = fitness,
                             maximize = True, max_val = 2)

max_attempts = 100
max_iters = math.inf

# run randomized hill climbing
start_time = time.time()
best_state, best_fitness, fitness_curve = mlrose.random_hill_climb(problem, restarts = 10, curve = True,
                                                                   max_attempts = 500, max_iters = max_iters,
                                                                   random_state = 42)
end_time = time.time()
rhc_time = end_time - start_time
print('The best state found for RHC is: ', best_state)
print('The fitness at the best state for RHC is: ', best_fitness)
print('run time for RHC is: ', rhc_time)

# run simulated annealing
start_time = time.time()
best_state_SA, best_fitness_SA, fitness_curve_SA = mlrose.simulated_annealing(problem, curve = True,
                                                                              max_attempts = max_attempts, 
                                                                              max_iters = max_iters,
                                                                              random_state = 42,
                                                                              schedule=mlrose.GeomDecay(init_temp = 100, decay=0.8, min_temp=1)
                                                                             )
end_time = time.time()
sa_time = end_time - start_time
print('The best state found for SA is: ', best_state_SA)
print('The fitness at the best state for SA is: ', best_fitness_SA)
print('run time for SA is: ', sa_time)

# run generic algorithm
start_time = time.time()
best_state_GA, best_fitness_GA, fitness_curve_GA = mlrose.genetic_alg(problem, curve = True,
                                                                      max_attempts = 5000, max_iters = max_iters,
                                                                      random_state = 42, pop_size=200,
                                                                      mutation_prob=0.001
                                                                     )
end_time = time.time()
ga_time = end_time - start_time
print('The best state found for GA is: ', best_state_GA)
print('The fitness at the best state for GA is: ', best_fitness_GA)
print('run time for GA is: ', ga_time)


# run MIMIC
start_time = time.time()
best_state_MM, best_fitness_MM, fitness_curve_MM = mlrose.mimic(problem, curve = True,
                                                                max_attempts = max_attempts, max_iters = max_iters,
                                                                random_state = 42
                                                                )
end_time = time.time()
mimic_time = end_time - start_time
print('The best state found for MIMIC is: ', best_state_MM)
print('The fitness at the best state for MIMIC is: ', best_fitness_MM)
print('run time for MIMIC is: ', mimic_time)

In [ ]:
## tune GA for four peaks
results = []

# max_attempts = 100
max_iters = math.inf

max_attempts_list = [100, 500, 1000, 5000]
pop_size_list = [100, 200, 500]
mutation_prob_list = [0.001, 0.005, 0.01, 0.05, 0.1]

# run generic algorithm
for pop in pop_size_list:
  for mutation in mutation_prob_list:
    for max_attempt in max_attempts_list:
        start_time = time.time()
        _, best_fitness_GA = mlrose.genetic_alg(problem, curve = False,
                                              max_attempts = max_attempt, max_iters = max_iters,
                                              random_state = 42, pop_size=pop,
                                              mutation_prob=mutation
                                              )

        end_time = time.time()
        use_time = end_time - start_time
        results.append([pop, mutation, max_attempt, 'GA', best_fitness_GA, use_time])
        print([pop, mutation, max_attempt, best_fitness_GA, use_time])

In [ ]:
df = pd.DataFrame(results, columns=["Population_Size", "Mutation_Probability","Max_Attempt","algorithm","best_fitness_value","runtime"])

In [ ]:
df.to_excel('/content/drive/MyDrive/CS7641A2/result_ga_4peaks.xlsx')

In [ ]:
print(len(fitness_curve))
print(len(fitness_curve_SA))
print(len(fitness_curve_GA))
print(len(fitness_curve_MM))

In [ ]:
## fill fitness curve after reach the max value with the max value
max_iters = 45000
fitness_RHC, len_RHC = fill_array(max_iters, fitness_curve)
fitness_SA, len_SA = fill_array(max_iters, fitness_curve_SA)
fitness_GA, len_GA = fill_array(max_iters, fitness_curve_GA)
fitness_MM, len_MM = fill_array(max_iters, fitness_curve_MM) 

In [ ]:
 # Plot Iterations vs Fitness
iterations = range(1, 45001)

plt.figure()

plt.plot(iterations, fitness_RHC, label='RHC', color='green')
plt.plot(iterations, fitness_SA, label='SA', color='red')
plt.plot(iterations, fitness_GA, label='GA', color='blue')
plt.plot(iterations, fitness_MM, label='MIMIC', color='orange')

plt.legend(loc="best")
plt.xlabel("Iterations")
plt.ylabel("Fitness")
plt.title('Fitness curve for four peak problem')

data = [('RHC', round(rhc_time, 5)), 
        ('SA', round(sa_time, 5)), 
        ('GA', round(ga_time, 5)), 
        ('MIMIC', round(mimic_time, 5))] 

df = pd.DataFrame(data, columns =['Algorithm', 'Time (s)']) 
print(df)

In [ ]:
##############################################
# define a max K color fitness function object
##############################################

## create a list of values from 0-99
elements = list(range(0, 100))
all_edges = list(itertools.combinations(elements, 2)) ## all edges between 0-99, 4950 edges
edges = random.sample(all_edges, 3000)


fitness = mlrose.MaxKColor(edges) 

# define a optimization problem object
problem = mlrose.DiscreteOpt(length = 100, fitness_fn = fitness,
                             maximize = True, max_val = 2)

max_attempts = 1000
max_iters = math.inf

# # run randomized hill climbing
start_time = time.time()
best_state, best_fitness, fitness_curve = mlrose.random_hill_climb(problem, restarts = 100, curve = True,
                                                                   max_attempts = 100, max_iters = max_iters,
                                                                   random_state = 42)
end_time = time.time()
rhc_time = end_time - start_time
print('The best state found for RHC is: ', best_state)
print('The fitness at the best state for RHC is: ', best_fitness)
print('run time for RHC is: ', rhc_time)

# run simulated annealing
start_time = time.time()
best_state_SA, best_fitness_SA, fitness_curve_SA = mlrose.simulated_annealing(problem, curve = True,
                                                                              max_attempts = max_attempts, 
                                                                              max_iters = max_iters,
                                                                              random_state = 42,
                                                                              schedule=mlrose.GeomDecay(init_temp = 100, decay=0.1, min_temp=1)
                                                                             )
end_time = time.time()
sa_time = end_time - start_time
print('The best state found for SA is: ', best_state_SA)
print('The fitness at the best state for SA is: ', best_fitness_SA)
print('run time for SA is: ', sa_time)

# run generic algorithm
start_time = time.time()
best_state_GA, best_fitness_GA, fitness_curve_GA = mlrose.genetic_alg(problem, curve = True,
                                                                      max_attempts = max_attempts, max_iters = max_iters,
                                                                      random_state = 42, 
                                                                      pop_size=200,
                                                                      mutation_prob=0.001
                                                                     )
end_time = time.time()
ga_time = end_time - start_time
print('The best state found for GA is: ', best_state_GA)
print('The fitness at the best state for GA is: ', best_fitness_GA)
print('run time for GA is: ', ga_time)


# run MIMIC
# start_time = time.time()
# best_state_MM, best_fitness_MM, fitness_curve_MM = mlrose.mimic(problem, curve = True,
#                                                                 max_attempts = max_attempts, max_iters = max_iters,
#                                                                 random_state = 42
#                                                                 )
# end_time = time.time()
# mimic_time = end_time - start_time
# print('The best state found for MIMIC is: ', best_state_MM)
# print('The fitness at the best state for MIMIC is: ', best_fitness_MM)
# print('run time for MIMIC is: ', mimic_time)

In [ ]:
print(len(fitness_curve))
print(len(fitness_curve_SA))
print(len(fitness_curve_GA))
print(len(fitness_curve_MM))

In [ ]:
## fill fitness curve after reach the max value with the max value
max_iters = 1000
fitness_RHC, len_RHC = fill_array(max_iters, fitness_curve)
fitness_SA, len_SA = fill_array(max_iters, fitness_curve_SA)
fitness_GA, len_GA = fill_array(max_iters, fitness_curve_GA)
fitness_MM, len_MM = fill_array(max_iters, fitness_curve_MM)

In [ ]:
 # Plot Iterations vs Fitness
iterations = range(1, 1001)

plt.figure()

plt.plot(iterations, fitness_RHC, label='RHC', color='green')
plt.plot(iterations, fitness_SA, label='SA', color='red')
plt.plot(iterations, fitness_GA, label='GA', color='blue')
plt.plot(iterations, fitness_MM, label='MIMIC', color='orange')

plt.legend(loc="best")
plt.xlabel("Iterations")
plt.ylabel("Fitness")
plt.title('Fitness curve for max K color problem')

data = [('RHC', round(rhc_time, 5)), 
        ('SA', round(sa_time, 5)), 
        ('GA', round(ga_time, 5)), 
        ('MIMIC', round(mimic_time, 5))] 

df = pd.DataFrame(data, columns =['Algorithm', 'Time (s)']) 
print(df)

In [ ]:
##############################################
# define a knapsack fitness function object
##############################################

weights = [random.randint(1,10) for i in range (0,100)]
values = [random.randint(1,10) for i in range (0,100)]
max_weight_pct = 0.6
fitness = mlrose.Knapsack(weights, values, max_weight_pct)

# define a optimization problem object
problem = mlrose.DiscreteOpt(length = 100, fitness_fn = fitness,
                             maximize = True, max_val = 2)

max_attempts = 100
max_iters = math.inf

# run randomized hill climbing
start_time = time.time()
best_state, best_fitness, fitness_curve = mlrose.random_hill_climb(problem, restarts = 1000, curve = True,
                                                                   max_attempts = 10, max_iters = max_iters,
                                                                   random_state = 42)
end_time = time.time()
rhc_time = end_time - start_time
print('The best state found for RHC is: ', best_state)
print('The fitness at the best state for RHC is: ', best_fitness)
print('run time for RHC is: ', rhc_time)

# run simulated annealing
start_time = time.time()
best_state_SA, best_fitness_SA, fitness_curve_SA = mlrose.simulated_annealing(problem, curve = True,
                                                                              max_attempts = max_attempts, 
                                                                              max_iters = max_iters,
                                                                              random_state = 42,
                                                                              schedule=mlrose.GeomDecay(init_temp = 100, decay=0.1, min_temp=1)
                                                                             )
end_time = time.time()
sa_time = end_time - start_time
print('The best state found for SA is: ', best_state_SA)
print('The fitness at the best state for SA is: ', best_fitness_SA)
print('run time for SA is: ', sa_time)

# run generic algorithm
start_time = time.time()
best_state_GA, best_fitness_GA, fitness_curve_GA = mlrose.genetic_alg(problem, curve = True,
                                                                      max_attempts = max_attempts, max_iters = max_iters,
                                                                      random_state = 42
                                                                     )
end_time = time.time()
ga_time = end_time - start_time
print('The best state found for GA is: ', best_state_GA)
print('The fitness at the best state for GA is: ', best_fitness_GA)
print('run time for GA is: ', ga_time)


# run MIMIC
start_time = time.time()
best_state_MM, best_fitness_MM, fitness_curve_MM = mlrose.mimic(problem, curve = True,
                                                                max_attempts = max_attempts, max_iters = max_iters,
                                                                random_state = 42
                                                                )
end_time = time.time()
mimic_time = end_time - start_time
print('The best state found for MIMIC is: ', best_state_MM)
print('The fitness at the best state for MIMIC is: ', best_fitness_MM)
print('run time for MIMIC is: ', mimic_time)

In [ ]:
print(len(fitness_curve))
print(len(fitness_curve_SA))
print(len(fitness_curve_GA))
print(len(fitness_curve_MM))

In [ ]:
## fill fitness curve after reach the max value with the max value
max_iters = 1500
fitness_RHC, len_RHC = fill_array(max_iters, fitness_curve)
fitness_SA, len_SA = fill_array(max_iters, fitness_curve_SA)
fitness_GA, len_GA = fill_array(max_iters, fitness_curve_GA)
fitness_MM, len_MM = fill_array(max_iters, fitness_curve_MM)

In [ ]:
 # Plot Iterations vs Fitness
iterations = range(1, 1501)

plt.figure()

plt.plot(iterations, fitness_RHC, label='RHC', color='green')
plt.plot(iterations, fitness_SA, label='SA', color='red')
plt.plot(iterations, fitness_GA, label='GA', color='blue')
plt.plot(iterations, fitness_MM, label='MIMIC', color='orange')

plt.legend(loc="best")
plt.xlabel("Iterations")
plt.ylabel("Fitness")
plt.title('Fitness curve for knapsack problem')

data = [('RHC', round(rhc_time, 5)), 
        ('SA', round(sa_time, 5)), 
        ('GA', round(ga_time, 5)), 
        ('MIMIC', round(mimic_time, 5))] 

df = pd.DataFrame(data, columns =['Algorithm', 'Time (s)']) 
print(df)